In [2]:
import pickle
import os
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

로그 스케일로 데이터 변환
- 지표간 차이
- 이전 STEP에서의 변화량

일반 스케일 데이터
- VOLUME/RSI 
- MA대신 MACD 지표 사용

가격데이터는 MID_PRICE만 사용

#### 데이터 가공

In [4]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\BTCUSD_1min.pkl', 'rb') as f:
    df_og = pickle.load(f)

In [5]:
df = df_og.loc['2022-11-14 05:15:00':]
df["mid_price"] = df['low']/2 + df['high']/2

df_og = df_og.loc['2022-11-14 05:15:00':]

In [6]:
df['log_returns'] = np.log(df['mid_price'] / df['mid_price'].shift(1))
df

,low,high,open,close,volume,mid_price,log_returns
time,,,,,,,
2022-11-14 05:15:00,15940.70,15951.72,15945.10,15941.61,37.486634,15946.210,NaN
2022-11-14 05:16:00,15936.86,15950.51,15943.82,15942.54,27.412980,15943.685,-0.000158
2022-11-14 05:17:00,15940.50,15957.37,15942.54,15950.75,30.118080,15948.935,0.000329
2022-11-14 05:18:00,15950.77,15969.00,15950.77,15968.94,16.315539,15959.885,0.000686
2022-11-14 05:19:00,15961.21,15973.60,15966.61,15965.72,43.274225,15967.405,0.000471
...,...,...,...,...,...,...,...
2023-11-14 05:11:00,36527.23,36538.29,36535.58,36535.43,0.551182,36532.760,0.000028
2023-11-14 05:12:00,36535.41,36538.32,36535.42,36536.52,1.442079,36536.865,0.000112
2023-11-14 05:13:00,36535.41,36549.62,36536.52,36535.42,5.654814,36542.515,0.000155


In [7]:
df['7min_MA'] = df['mid_price'].rolling(window=7).mean()
df['25min_MA'] = df['mid_price'].rolling(window=25).mean()
df['99min_MA'] = df['mid_price'].rolling(window=90).mean()
df['300min_MA'] = df['mid_price'].rolling(window=300).mean()
df['900min_MA'] = df['mid_price'].rolling(window=900).mean()


df['7_ema'] = df['mid_price'].ewm(span=7, adjust=False).mean()
df['25_ema'] = df['mid_price'].ewm(span=25, adjust=False).mean()
df['99_ema'] = df['mid_price'].ewm(span=90,adjust=False).mean()
df['300_ema'] = df['mid_price'].ewm(span=300,adjust=False).mean()
df['900_ema'] = df['mid_price'].ewm(span=900,adjust=False).mean()


df['log_7min_MA'] = df['log_returns'].rolling(window=7).mean()
df['log_25min_MA'] = df['log_returns'].rolling(window=25).mean()
df['log_99min_MA'] = df['log_returns'].rolling(window=90).mean()
df['log_300min_MA'] = df['log_returns'].rolling(window=300).mean()
df['log_900min_MA'] = df['log_returns'].rolling(window=900).mean()


df['log_7_ema'] = df['log_returns'].ewm(span=7, adjust=False).mean()
df['log_25_ema'] = df['log_returns'].ewm(span=25, adjust=False).mean()
df['log_99_ema'] = df['log_returns'].ewm(span=90,adjust=False).mean()
df['log_300_ema'] = df['log_returns'].ewm(span=300,adjust=False).mean()
df['log_900_ema'] = df['log_returns'].ewm(span=900,adjust=False).mean()

In [8]:
def calculate_macd(df, short_window=12, long_window=26, signal_window=9):
    # 단기 및 장기 지수 이동 평균 계산
    df['Short_EMA'] = df['mid_price'].ewm(span=short_window, adjust=False).mean()
    df['Long_EMA'] = df['mid_price'].ewm(span=long_window, adjust=False).mean()

    # MACD 계산
    df['MACD'] = df['Short_EMA'] - df['Long_EMA']

    # 신호선 계산
    df['Signal_Line'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()

    return df[['MACD', 'Signal_Line']]

macd_df = calculate_macd(df)

macd_df

,MACD,Signal_Line
time,,
2022-11-14 05:15:00,0.000000,0.000000
2022-11-14 05:16:00,-0.201425,-0.040285
2022-11-14 05:17:00,0.061863,-0.019855
2022-11-14 05:18:00,1.140942,0.212304
2022-11-14 05:19:00,2.573258,0.684495
...,...,...
2023-11-14 05:11:00,11.503282,12.163212
2023-11-14 05:12:00,11.388726,12.008315
2023-11-14 05:13:00,11.619901,11.930632


In [9]:
df['obv'] = [0 for x in range(df.shape[0])]

for i in tqdm(range(1, len(df))):
    if df['close'][i] > df['close'][i - 1]:
        df['obv'][i] = df['obv'][i - 1] + df['volume'][i]
    elif df['close'][i] < df['close'][i - 1]:
        df['obv'][i] = df['obv'][i - 1] - df['volume'][i]
    else:
        df['obv'][i] = df['obv'][i - 1]

100%|██████████| 525254/525254 [00:27<00:00, 18833.06it/s]


In [10]:
df['obv'] /= 5000

In [11]:
df = df.dropna(subset=['mid_price'])

# Calculate price changes
delta = df['mid_price'].diff(1)

# Define the time period for RSI (e.g., 14)
rsi_period = 14

# Calculate gains and losses
gains = delta.where(delta > 0, 0)
losses = -delta.where(delta < 0, 0)

# Calculate average gains and losses
avg_gain = gains.rolling(window=rsi_period, min_periods=1).mean()
avg_loss = losses.rolling(window=rsi_period, min_periods=1).mean()

# Calculate relative strength (RS)
rs = avg_gain / avg_loss

# Calculate RSI
df['rsi'] = (100 - (100 / (1 + rs)))/100

In [12]:
df['vma'] = df['volume'].rolling(window=20).mean()

In [13]:
df.head()

,low,high,open,close,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,...,log_99_ema,log_300_ema,log_900_ema,Short_EMA,Long_EMA,MACD,Signal_Line,obv,rsi,vma
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 05:15:00,15940.70,15951.72,15945.10,15941.61,37.486634,15946.210,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15946.210000,15946.210000,0.000000,0.000000,0.000000,NaN,NaN
2022-11-14 05:16:00,15936.86,15950.51,15943.82,15942.54,27.412980,15943.685,-0.000158,NaN,NaN,NaN,...,-0.000158,-0.000158,-0.000158,15945.821538,15946.022963,-0.201425,-0.040285,0.005483,0.000000,NaN
2022-11-14 05:17:00,15940.50,15957.37,15942.54,15950.75,30.118080,15948.935,0.000329,NaN,NaN,NaN,...,-0.000148,-0.000155,-0.000157,15946.300533,15946.238669,0.061863,-0.019855,0.011506,0.675241,NaN
2022-11-14 05:18:00,15950.77,15969.00,15950.77,15968.94,16.315539,15959.885,0.000686,NaN,NaN,NaN,...,-0.000129,-0.000150,-0.000155,15948.390451,15947.249509,1.140942,0.212304,0.014769,0.865154,NaN
2022-11-14 05:19:00,15961.21,15973.60,15966.61,15965.72,43.274225,15967.405,0.000471,NaN,NaN,NaN,...,-0.000116,-0.000145,-0.000154,15951.315766,15948.742508,2.573258,0.684495,0.006114,0.903791,NaN


In [14]:
df.columns

Index(['low', 'high', 'open', 'close', 'volume', 'mid_price', 'log_returns',
       '7min_MA', '25min_MA', '99min_MA', '300min_MA', '900min_MA', '7_ema',
       '25_ema', '99_ema', '300_ema', '900_ema', 'log_7min_MA', 'log_25min_MA',
       'log_99min_MA', 'log_300min_MA', 'log_900min_MA', 'log_7_ema',
       'log_25_ema', 'log_99_ema', 'log_300_ema', 'log_900_ema', 'Short_EMA',
       'Long_EMA', 'MACD', 'Signal_Line', 'obv', 'rsi', 'vma'],
      dtype='object')

In [15]:
def pct_change_with_zero_handling(df):
    # Step 1: Calculate regular percentage change
    pct_change_result = df.pct_change()
    inf_mask = np.isinf(pct_change_result)
    pct_change_result[inf_mask] = 0
    return pct_change_result

price_col =['volume', 'mid_price', 'log_returns',
       '7min_MA', '25min_MA', '99min_MA', '300min_MA', '900min_MA', '7_ema',
       '25_ema', '99_ema', '300_ema', '900_ema', 'Short_EMA',
       'Long_EMA', 'vma']

df[price_col] = pct_change_with_zero_handling(df[price_col])

np.isinf(df[price_col].values).sum()

0

In [16]:
df["vol_pct"] = df[['volume']].pct_change()

In [17]:
df['MACD-SIGNAL'] = df['MACD'] - df['Signal_Line']

In [18]:
df_final = df.drop(['Short_EMA', "Long_EMA", 'high', 'low', 'open', 'close'], axis=1)
df_final = df_final.dropna()
df_final.head()

,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,...,log_99_ema,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 20:15:00,0.401930,0.001804,2.406179,0.000063,-0.000455,-0.000250,-0.000080,0.000019,0.000373,-0.000146,...,-0.000228,-0.000111,-0.000055,-41.318663,-47.808444,-0.039246,0.444260,-0.111538,-4.903050,6.489781
2022-11-14 20:16:00,-0.322941,0.000225,-0.875063,0.000342,-0.000482,-0.000243,-0.000081,0.000020,0.000336,-0.000118,...,-0.000218,-0.000109,-0.000054,-37.196986,-45.686152,-0.031542,0.447616,-0.048051,-1.803474,8.489166
2022-11-14 20:17:00,0.047696,0.001045,3.640008,0.000262,-0.000404,-0.000231,-0.000075,0.000020,0.000513,-0.000028,...,-0.000190,-0.000101,-0.000052,-32.190122,-42.986946,-0.023471,0.427829,-0.003950,-1.147692,10.796824
2022-11-14 20:18:00,0.751147,0.000294,-0.718221,0.000422,-0.000253,-0.000242,-0.000069,0.000020,0.000459,-0.000003,...,-0.000180,-0.000098,-0.000051,-27.518817,-39.893321,-0.009338,0.437425,0.011163,14.748759,12.374503
2022-11-14 20:19:00,-0.154670,0.000703,1.388568,0.000675,-0.000139,-0.000222,-0.000065,0.000020,0.000520,0.000051,...,-0.000160,-0.000093,-0.000050,-22.633165,-36.441289,0.002610,0.524746,0.003284,-1.205912,13.808125


In [19]:
import pandas as pd
pd.set_option('display.max_columns', None)

print(df_final.shape)
df_final.describe()

(524117, 30)


,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,99_ema,300_ema,900_ema,log_7min_MA,log_25min_MA,log_99min_MA,log_300min_MA,log_900min_MA,log_7_ema,log_25_ema,log_99_ema,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL
count,524117.000000,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,524117.000000,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,524117.000000,5.241170e+05,5.241170e+05,5.241170e+05,5.241170e+05,524117.000000,524117.000000,524117.000000,524117.000000,524117.000000,524117.000000,524117.000000
mean,0.738323,1.665706e-06,4.692454e+06,1.571940e-06,1.549204e-06,1.520346e-06,1.511122e-06,1.513483e-06,0.000002,1.548892e-06,1.526689e-06,1.512849e-06,1.521889e-06,1.549037e-06,1.542708e-06,1.518643e-06,1.510606e-06,1.513527e-06,0.000002,1.542469e-06,1.524984e-06,1.512274e-06,1.503535e-06,0.269759,0.269346,-0.606259,0.503153,0.003606,-2.955052,0.000413
std,7.661235,4.777686e-04,8.329837e+09,2.140164e-04,1.141428e-04,5.947482e-05,3.291977e-05,1.913589e-05,0.000211,1.135980e-04,5.966224e-05,3.278586e-05,1.933084e-05,2.140150e-04,1.141320e-04,5.946786e-05,3.291670e-05,1.912341e-05,0.000211,1.135857e-04,5.964222e-05,3.275164e-05,1.927138e-05,17.097368,16.107529,2.570343,0.190830,0.097846,868.062074,5.129154
min,-0.998152,-3.383133e-02,-2.097155e+12,-1.087993e-02,-3.116766e-03,-9.335592e-04,-2.972316e-04,-1.236377e-04,-0.013976,-5.183336e-03,-1.663310e-03,-5.505826e-04,-2.076951e-04,-1.109981e-02,-3.224901e-03,-9.679269e-04,-3.094243e-04,-1.290082e-04,-0.014350,-5.349346e-03,-1.727730e-03,-5.743286e-04,-2.178140e-04,-460.909365,-341.306509,-4.960160,0.000000,-0.844168,-546203.990281,-225.095058
25%,-0.452078,-1.610431e-04,-1.689928e+00,-7.295931e-05,-3.767068e-05,-1.895013e-05,-1.052240e-05,-6.765792e-06,-0.000069,-3.597800e-05,-1.771510e-05,-9.627236e-06,-6.002677e-06,-7.295595e-05,-3.767102e-05,-1.894684e-05,-1.052311e-05,-6.764022e-06,-0.000069,-3.596046e-05,-1.768979e-05,-9.602874e-06,-5.976118e-06,-5.168584,-4.841970,-2.602108,0.363514,-0.027490,-2.644569,-1.766587
50%,-0.018761,1.675533e-07,-7.457784e-01,8.843221e-07,5.825485e-07,4.066774e-07,2.342069e-07,2.546229e-07,0.000001,7.264216e-07,4.207057e-07,2.727754e-07,1.302828e-07,8.844628e-07,5.824623e-07,4.067620e-07,2.342728e-07,2.544959e-07,0.000001,7.473933e-07,4.399894e-07,2.929686e-07,1.474810e-07,0.087208,0.066508,-0.522720,0.501120,-0.000549,-1.261113,-0.004252
75%,0.781987,1.636915e-04,2.270586e-01,7.576633e-05,3.961699e-05,2.065929e-05,1.136939e-05,7.649257e-06,0.000073,3.819757e-05,1.936972e-05,1.042746e-05,6.874105e-06,7.576535e-05,3.961770e-05,2.066199e-05,1.136922e-05,7.652422e-06,0.000073,3.822401e-05,1.939227e-05,1.045486e-05,6.886562e-06,5.512914,5.208736,1.661814,0.642865,0.025903,-0.518503,1.770085
max,4796.874745,3.336840e-02,2.342612e+12,9.763860e-03,3.085761e-03,1.049758e-03,3.681025e-04,1.553923e-04,0.013695,5.004469e-03,1.725989e-03,6.069209e-04,2.380470e-04,9.586431e-03,2.998128e-03,1.039921e-03,3.559441e-04,1.501581e-04,0.013352,4.852726e-03,1.663965e-03,5.814832e-04,2.264805e-04,534.245074,399.719208,4.608425,1.000000,9.349762,149178.629496,236.834237


In [20]:
df_final.to_csv('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\df_final.csv')

In [21]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\train_data\\df_final.pkl', 'wb') as f:
    pickle.dump(df_final, f)


#### df_final의 raw 데이터도 저장

In [22]:
df_og.loc[df_final.index]
df_og['mid_price'] = df_og['high']/2 + df_og['low']/2
df_og['pct_change'] = df_og['mid_price'].pct_change()
df_final_raw = df_og.drop(['low', 'high', 'open', 'close'], axis=1)
df_final_raw = df_final_raw.loc[df_final.index]
df_final_raw

,volume,mid_price,pct_change
time,,,
2022-11-14 20:15:00,56.890409,16233.560,0.001804
2022-11-14 20:16:00,38.518180,16237.215,0.000225
2022-11-14 20:17:00,40.355329,16254.185,0.001045
2022-11-14 20:18:00,70.668109,16258.970,0.000294
2022-11-14 20:19:00,59.737850,16270.405,0.000703
...,...,...,...
2023-11-14 05:11:00,0.551182,36532.760,0.000028
2023-11-14 05:12:00,1.442079,36536.865,0.000112
2023-11-14 05:13:00,5.654814,36542.515,0.000155


In [23]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\train_data\\df_final_raw.pkl', 'wb') as f:
    pickle.dump(df_final_raw, f)

#### 학습용데이터(이 데이터를 적재하는 것보다 학습단계에서 전처리하는게 빨라서 사용하지 않음)

In [20]:
import numpy as np
from tqdm import tqdm


# 주어진 데이터프레임
sequence_length = 64
n_samples = len(df_final) - sequence_length + 1

# Extract the data as a NumPy array
data_array = df_final.to_numpy()

# Create an empty array to store the input sequences
input_sequences = np.empty((n_samples, (df_final.shape[1]), sequence_length))

for i in tqdm(range(n_samples)):
    input_seq = data_array[i:i + sequence_length, :].T  # Transpose to have (24, 60) shape
    input_sequences[i] = input_seq

print(input_sequences.shape)

100%|██████████| 524292/524292 [00:02<00:00, 220845.89it/s]

(524292, 30, 64)


In [21]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\binance_data.pkl', 'wb') as f:
    pickle.dump(input_sequences, f)

In [20]:
with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\binance_data.pkl', 'rb') as f:
    df = pickle.load(f)


df.shape

(524292, 30, 64)

In [22]:
df = df.dropna()
df.shape

AttributeError: 'numpy.ndarray' object has no attribute 'dropna'